In [1]:
from myphdlib.interface.mlati import MlatiSession
import matplotlib.pylab as plt
import os
import re
import numpy as np
import nptracer as npt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.ndimage.filters import gaussian_filter1d
import warnings
warnings.filterwarnings("ignore")
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
import pathlib as pl
from sklearn.linear_model import LinearRegression, Ridge, PoissonRegressor
from scipy import signal, stats
from myphdlib.figures.bootstrap import BootstrappedSaccadicModulationAnalysis
import math
#DOING PCA ON PREMOTOR, SPLIT VIS NEURONS INTO 3 CATEGORIES, DO REGRESSION, AVG THE WEIGHTS SO 1 VALUE PER TIME POINT PER VIS NEURON TYPE
#THEN PLOT AS A FXN OF TIME - LINE FOR EACH CATEGORY

/tmp/ipykernel_15747/2363756313.py:10: DeprecationWarning: Please import `gaussian_filter1d` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import gaussian_filter1d


In [2]:
sessionsMlati = list()
sessionsMlatiList = ['/media/onecore/Seagate Portable Drive/2023-07-24/mlati9',
                     '/media/onecore/Seagate Portable Drive/2023-07-25/mlati10',
                     '/media/onecore/Seagate Portable Drive/2023-07-26/mlati9',
                     '/media/onecore/Seagate Portable Drive/2023-07-27/mlati10',
                     '/media/onecore/Seagate Portable Drive/2023-07-28/mlati9',
                     '/media/onecore/Seagate Portable Drive/2023-07-31/mlati10',
                     '/media/onecore/Seagate Portable Drive/2023-08-01/mlati10'
                  ]
                  
for session in sessionsMlatiList:
    sessionToAppend = MlatiSession(session)
    sessionsMlati.append(sessionToAppend)

In [3]:
keys = list()
for session in sessionsMlati:
    keys.append(str(session.date))
premotorUnitsZeta = {key:[] for key in keys}
for session in sessionsMlati:
    spikeClustersFile = session.home.joinpath('ephys/sorting/manual/spike_clusters.npy')
    uniqueSpikeClusters = np.unique(np.load(spikeClustersFile))
    zetaNasal = session.load('zeta/saccade/nasal/p')
    zetaTemporal = session.load('zeta/saccade/temporal/p')
    ampCutoff = session.load('metrics/ac')
    presenceRatio = session.load('metrics/pr')
    firingRate = session.load('metrics/fr')
    isiViol = session.load('metrics/rpvr')
    qualityLabels = session.load('metrics/ql')
    for index, pVal in enumerate(zetaNasal):
        pNasal = pVal
        pTemporal = zetaTemporal[index]
        if pNasal < pTemporal:
            p = pNasal
        elif pTemporal < pNasal:
            p = pTemporal
        if p < 0.05:
            if qualityLabels is not None and qualityLabels[index] in (0, 1):
                    continue
            if ampCutoff[index] <= 0.1:
                if presenceRatio[index] >= 0.9:
                    if firingRate[index] >= 0.2:
                        if isiViol[index] <= 0.5:
                            date = str(session.date)
                            unit = uniqueSpikeClusters[index]
                            premotorUnitsZeta[date].append(unit)

In [4]:
keys = list()
for session in sessionsMlati:
    keys.append(str(session.date))
visualUnitsZeta = {key:[] for key in keys}
for session in sessionsMlati:
    spikeClustersFile = session.home.joinpath('ephys/sorting/manual/spike_clusters.npy')
    uniqueSpikeClusters = np.unique(np.load(spikeClustersFile))
    zetaLeft = session.load('zeta/probe/left/p')
    zetaRight = session.load('zeta/probe/right/p')
    ampCutoff = session.load('metrics/ac')
    presenceRatio = session.load('metrics/pr')
    firingRate = session.load('metrics/fr')
    isiViol = session.load('metrics/rpvr')
    qualityLabels = session.load('metrics/ql')
    for index, pVal in enumerate(zetaLeft):
        pLeft = pVal
        pRight = zetaRight[index]
        if pLeft < pRight:
            p = pLeft
        elif pRight < pLeft:
            p = pRight
        if p < 0.05:
            if qualityLabels is not None and qualityLabels[index] in (0, 1):
                    continue
            if ampCutoff[index] <= 0.1:
                if presenceRatio[index] >= 0.9:
                    if firingRate[index] >= 0.2:
                        if isiViol[index] <= 0.5:
                            date = str(session.date)
                            unit = uniqueSpikeClusters[index]
                            visualUnitsZeta[date].append(unit)

In [5]:
perisaccadicProbes = {key:[] for key in keys}
probeDirections = {key:[] for key in keys}
for session in sessionsMlati:
    probeTimestamps = session.probeTimestamps
    saccadeTimestamps = session.saccadeTimestamps[:, 0]
    probeDirectionsSess = session.gratingMotionDuringProbes
    for i, probe in enumerate(probeTimestamps):
        relative = abs(saccadeTimestamps - probe)
        if any(relative < 0.5):
            perisaccadicProbes[str(session.date)].append(probe)
            probeDirections[str(session.date)].append(probeDirectionsSess[i])

In [6]:
sacPreProbe = list()
sacPreProbeDir = list()
sacDir = list()
sac01 = list()
sac02 = list()
sac03 = list()
sac04 = list()
sac05 = list()
sac06 = list()
sac07 = list()
sac08 = list()
sac09 = list()
sac10 = list()
#for session in sessionsMlati:
session = sessionsMlati[0]
saccadeTimestamps = session.saccadeTimestamps[:, 0]
sacAmpList = list()
#sacDirections = session.gratingMotionDuringSaccades
#sacDirections = session.load('saccades/predicted/left/labels')
for i, probe in enumerate(perisaccadicProbes[str(session.date)]):
    relative = saccadeTimestamps - probe
    closestInd = int(np.where(abs(relative) == np.nanmin(abs(relative)))[0])
    closestSac = saccadeTimestamps[closestInd]
   # sacAmp = ampList[closestInd]
    #sacDirTmp = (sacDirections[closestInd])
    if closestSac < probe:
        sacPreProbe.append(probe)
        sacPreProbeDir.append(probeDirections[str(session.date)][i])
        #sacDir.append(sacDirTmp)
        #sacAmpList.append(sacAmp)
        diff = probe - closestSac
        if diff <= 0.1:
            sac01.append(probe)
        else:
            if diff <= 0.2:
                sac02.append(probe)
            else:
                if diff <= 0.3:
                    sac03.append(probe)
                else:
                    if diff <= 0.4:
                        sac04.append(probe)
                    else:
                        if diff <= 0.5:
                            sac05.append(probe)
    else:
        diff = closestSac - probe
        if diff <= 0.1:
            sac06.append(probe)
        else:
            if diff <= 0.2:
                sac07.append(probe)
            else:
                if diff <= 0.3:
                    sac08.append(probe)
                else:
                    if diff <= 0.4:
                        sac09.append(probe)
                    else:
                        if diff <= 0.5:
                            sac10.append(probe)

In [7]:
agree = list()
unitsP = premotorUnitsZeta[str(session.date)]
unitsV = visualUnitsZeta[str(session.date)]
for u in unitsP:
    if u in unitsV:
        agree.append(u)
trials = list()
#visualArray = np.zeros((len(visualUnitsZeta[str(session.date)]) - len(agree), len(perisaccadicProbes[str(session.date)])))
#premotorArray = np.zeros((len(premotorUnitsZeta[str(session.date)]) - len(agree), len(perisaccadicProbes[str(session.date)])))
for probe in sacPreProbe:
    relative = saccadeTimestamps - probe
    closestInd = np.where(abs(relative) == np.nanmin(abs(relative)))[0]
    closestSac = saccadeTimestamps[closestInd]
    unitInd = 0
    unitArray = np.zeros((len(premotorUnitsZeta[str(session.date)]) - len(agree)))
    for unit in session.population:
        if unit.cluster in agree:
            continue
        if unit.cluster in premotorUnitsZeta[str(session.date)]:
            spikeTimes = unit.timestamps
            start = probe - 0.3
            end = probe
            mask = np.logical_and(spikeTimes > start, spikeTimes < end)
            activity = len(spikeTimes[mask])/0.3
            unitArray[unitInd] = activity
            unitInd = unitInd + 1
    trials.append(unitArray)

In [8]:
def z_score(X):
    # X: ndarray, shape (n_features, n_samples)
    ss = StandardScaler(with_mean=True, with_std=True)
    Xz = ss.fit_transform(X.T).T
    return Xz
shade_alpha      = 0.2
lines_alpha      = 0.8
pal = sns.color_palette('husl', 2)
%config InlineBackend.figure_format = 'svg'

def add_stim_to_plot(ax):
    ax.axvspan(start_stim, end_stim, alpha=shade_alpha,
               color='gray')
    ax.axvline(start_stim, alpha=lines_alpha, color='gray', ls='--')
    ax.axvline(end_stim, alpha=lines_alpha, color='gray', ls='--')
    
def add_orientation_legend(ax):
    custom_lines = [Line2D([0], [0], color=pal[k], lw=4) for
                    k in range(len(trial_types))]
    labels = ['{}'.format(t) for t in trial_types]
    ax.legend(custom_lines, labels,
              frameon=False, loc='center left', bbox_to_anchor=(1, 0.5))
    plt.tight_layout(rect=[0,0,0.9,1])

In [17]:
trials[0].shape

(30,)

In [14]:
#trial concatenated PCA
trial_types = [-1, 1]
Xl = np.hstack(trials)
print(Xl.shape)
Xl = z_score(Xl)
pca = PCA(n_components=15)
Xl_p = pca.fit_transform(Xl.T).T
gt = {comp : {t_type : [] for t_type in trial_types} for comp in range(n_components)}
for comp in range(n_components):
    for i, t_type in enumerate(trial_type):
        t = Xl_p[comp, trial_size * i: trial_size * (i + 1)]
        gt[comp][t_type].append(t)
for comp in range(n_components):
    for t_type in trial_types:
        gt[comp][t_type] = np.vstack(gt[comp][t_type])


(23130,)


ValueError: Expected 2D array, got 1D array instead:
array=[3.33333333 6.66666667 0.         ... 0.         3.33333333 6.66666667].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.